# Extração, transformação e carregamento (ETL) dos dados do pdf da Produção de Cupuaçu

## Leitura do arquivo PDF 

In [15]:
# Bibliotecas necessárias
import pdfplumber
import pandas as pd
import numpy as np
import time
from geopy.geocoders import Nominatim

# Caminho do arquivo PDF
file_path = "Cultura do Cupuaçu - jan a dez 2023.pdf"

# Abrir o PDF com pdfplumber
with pdfplumber.open(file_path) as pdf:
    # Extrair a primeira tabela da primeira página
    table = pdf.pages[0].extract_table()


In [16]:
# Convertendo a tabela extraída para um DataFrame
df = pd.DataFrame(table[1:], columns=table[0])

# Substituir 'none' como string e None (valores nulos) por vazio em todas as colunas
df = df.applymap(lambda x: np.nan if x == 'none' or pd.isna(x) else x)

df

,Unidade local / Sub-região,ATER,None,None,None,ESTIMATIVA DO MUNICÍPIO,None,None,None
0,NaN,Nº de\nBeneficiários,Área (ha),NaN,Produção (mil\nfrutos),Nº de\nBeneficiários,Área Estimada (ha),NaN,Produção (mil\nfrutos)
1,NaN,NaN,Plantada,Colhida,NaN,NaN,Plantada,Em produção,NaN
2,ALTO SOLIMÕES,55,"17,10","15,71","23,64",300,"120,48","112,23","159,58"
3,Amaturá,3,"2,28","0,98","1,96",80,"12,00","4,75","7,60"
4,Atalaia do Norte,1,"0,10","0,10","0,10",70,"35,00","35,00","28,00"
...,...,...,...,...,...,...,...,...,...
74,Nhamundá,,,,,,,,
75,S. Sebastião do Uatumã,1,"0,50","0,50","0,75",16,"9,50","9,50","11,40"
76,Urucará,3,"4,50","4,50","9,00",45,"32,00","32,00","51,20"
77,Parintins,3,"1,30","1,30","2,60",57,"30,50","30,50","48,80"


## Tratamento de Dados 

In [17]:
# Preenchendo (filldown) valores nulos na linha 1 com valores da linha 0
df = df.fillna(method='ffill')

# Removendo a linha 1 após o preenchimento (filldown)
df = df.drop(0).reset_index(drop=True)

# Setando a primeira linha como cabeçalho
df.columns = df.iloc[0]
df = df.drop(0).reset_index(drop=True)

In [18]:
# Verificando se há colunas com nomes duplicados
duplicated_columns = df.columns[df.columns.duplicated()].unique()
duplicated_columns


Index(['Nº de\nBeneficiários', 'Plantada', 'Produção (mil\nfrutos)'], dtype='object', name=0)

In [19]:
# Renomeando colunas duplicadas de forma sequencial (Nome1, Nome2, etc.)
columns = df.columns.tolist()
new_columns = []

# Dicionário para rastrear a contagem de colunas com o mesmo nome
col_count = {}

for idx, col in enumerate(columns):
    # Se o nome da coluna já apareceu antes, incrementa o número sequencial
    if col in col_count:
        col_count[col] += 1
        new_columns.append(f"{col}_{col_count[col]}")
    else:
        col_count[col] = 0
        new_columns.append(col)

# Atribuindo os novos nomes de colunas ao DataFrame
df.columns = new_columns

# Renomeando a primeira coluna para 'Unidade local'
df.columns.values[0] = 'Unidade local'

# Exibindo os novos nomes das colunas
df.columns


Index(['Unidade local', 'Nº de\nBeneficiários', 'Plantada', 'Colhida',
       'Produção (mil\nfrutos)', 'Nº de\nBeneficiários_1', 'Plantada_1',
       'Em produção', 'Produção (mil\nfrutos)_1'],
      dtype='object')

In [20]:
# Remover espaços extras das colunas
df.columns = df.columns.str.strip()

# Renomeando as colunas de ater e estimativa
df.columns.values[0] = "Unidade local"
df.columns.values[1] = "Nº de Beneficiários"
df.columns.values[2] = "Plantada (ha)"
df.columns.values[3] = "Colhida (ha)"
df.columns.values[4] = "Produção (mil frutos)"
df.columns.values[5] = "Nº de Beneficiários (est.)"
df.columns.values[6] = "Plantada (ha) (est.)"
df.columns.values[7] = "Em produção (ha) (est.)"
df.columns.values[8] = "Produção (mil frutos) (est.)"



In [21]:
# Excluindo as linhas onde as colunas especificadas têm valores ausentes
df[['Nº de Beneficiários', 'Plantada (ha)', 'Colhida (ha)', 'Produção (mil frutos)']] = df[['Nº de Beneficiários', 'Plantada (ha)', 'Colhida (ha)', 'Produção (mil frutos)']
].replace("", np.nan)

df = df.dropna(subset=['Nº de Beneficiários', 'Plantada (ha)', 'Colhida (ha)', 'Produção (mil frutos)'])


In [22]:
# Filtrando as sub-regiões indesejadas
sub_regioes_excluidas = [
    "ALTO RIO NEGRO", "ALTO SOLIMÕES", "BAIXO AMAZONAS", "JURUÁ", 
    "JUTAÍ/SOLIMÕES/JURUÁ", "MADEIRA", "MÉDIO AMAZONAS", "PURÚS", 
    "RIO NEGRO/SOLIMÕES", "T O T A L"
]

df = df[~df["Unidade local"].isin(sub_regioes_excluidas)]


In [23]:
# Lista de colunas que contêm valores decimais com vírgulas
decimal_columns = [
    "Plantada (ha)", "Colhida (ha)", "Produção (mil frutos)", 
    "Plantada (ha) (est.)", "Em produção (ha) (est.)", "Produção (mil frutos) (est.)"
]

# Converter as colunas substituindo vírgula por ponto antes de converter para float
for col in decimal_columns:
    df[col] = df[col].str.replace(',', '.').astype(float)

# Alterando o tipo das colunas restantes
df = df.astype({
    "Unidade local": "str",
    "Nº de Beneficiários": "Int64",
    "Nº de Beneficiários (est.)": "Int64"
})

In [24]:
df.dtypes

Unidade local                    object
Nº de Beneficiários               Int64
Plantada (ha)                   float64
Colhida (ha)                    float64
Produção (mil frutos)           float64
Nº de Beneficiários (est.)        Int64
Plantada (ha) (est.)            float64
Em produção (ha) (est.)         float64
Produção (mil frutos) (est.)    float64
dtype: object

In [25]:
df

,Unidade local,Nº de Beneficiários,Plantada (ha),Colhida (ha),Produção (mil frutos),Nº de Beneficiários (est.),Plantada (ha) (est.),Em produção (ha) (est.),Produção (mil frutos) (est.)
1,Amaturá,3,2.28,0.98,1.96,80,12.00,4.75,7.60
2,Atalaia do Norte,1,0.10,0.10,0.10,70,35.00,35.00,28.00
3,Benjamin Constant,3,3.50,3.40,8.50,40,28.00,27.00,54.00
4,São Paulo de Olivença,9,1.85,1.85,3.70,12,12.00,12.00,19.20
6,Tabatinga,39,9.38,9.38,9.38,39,9.38,9.38,9.38
9,Alvarães,7,3.00,3.00,4.50,10,4.00,4.00,4.80
10,Fonte Boa,7,1.30,1.30,2.60,8,1.50,1.50,2.60
11,Japurá,9,2.60,0.60,0.60,28,9.60,7.60,6.08
12,Juruá,31,5.94,5.94,11.88,32,8.00,8.00,12.80
13,Jutaí,5,4.70,3.70,11.10,80,45.00,44.00,105.60


### Procurando por meio geocoder Nominatim as coordenadas de cada localidade

In [ ]:
# Adicionando as colunas de coordenadas geográficas

# Inicializar o geolocalizador com o Nominatim
geolocator = Nominatim(user_agent="cupuaçu-geolocator")

# Função para obter as coordenadas de uma localidade
def get_coordinates(local):
    try:
        location = geolocator.geocode(local + ", Amazonas, Brasil")
        if location:
            return location.latitude, location.longitude
        else:
            return np.nan, np.nan
    except Exception as e:
        print(f"Erro ao processar {local}: {e}")
        return np.nan, np.nan

# Criando as colunas de latitude e longitude
df['Latitude'] = np.nan
df['Longitude'] = np.nan

# Iterando pelas localidades (Unidade local) e obtendo as coordenadas
for idx, row in df.iterrows():
    local = row['Unidade local']
    
    # Obter coordenadas
    lat, lon = get_coordinates(local)
    
    # Atualizar o DataFrame com os valores de latitude e longitude
    df.at[idx, 'Latitude'] = lat
    df.at[idx, 'Longitude'] = lon
    
    # Timer de 1 segundo entre as requisições
    time.sleep(1)

In [28]:
df

,Unidade local,Nº de Beneficiários,Plantada (ha),Colhida (ha),Produção (mil frutos),Nº de Beneficiários (est.),Plantada (ha) (est.),Em produção (ha) (est.),Produção (mil frutos) (est.),Latitude,Longitude
1,Amaturá,3,2.28,0.98,1.96,80,12.00,4.75,7.60,-3.364051,-68.199469
2,Atalaia do Norte,1,0.10,0.10,0.10,70,35.00,35.00,28.00,-4.365855,-70.190874
3,Benjamin Constant,3,3.50,3.40,8.50,40,28.00,27.00,54.00,-4.378275,-70.029060
4,São Paulo de Olivença,9,1.85,1.85,3.70,12,12.00,12.00,19.20,-3.465063,-68.946846
6,Tabatinga,39,9.38,9.38,9.38,39,9.38,9.38,9.38,-4.232735,-69.926337
9,Alvarães,7,3.00,3.00,4.50,10,4.00,4.00,4.80,-3.215579,-64.814833
10,Fonte Boa,7,1.30,1.30,2.60,8,1.50,1.50,2.60,-2.514746,-66.094842
11,Japurá,9,2.60,0.60,0.60,28,9.60,7.60,6.08,-1.880847,-66.996766
12,Juruá,31,5.94,5.94,11.88,32,8.00,8.00,12.80,-3.474667,-66.062234
13,Jutaí,5,4.70,3.70,11.10,80,45.00,44.00,105.60,-2.747148,-66.772126


In [29]:
# Caminho onde o arquivo será salvo
output_file = "producao_cupuaçu_com_coordenadas.xlsx"

# Salvando o DataFrame em um arquivo Excel (.xlsx)
df.to_excel(output_file, index=False)

# Mensagem de sucesso
print(f"Arquivo salvo com sucesso: {output_file}")


Arquivo salvo com sucesso: producao_cupuaçu_com_coordenadas.xlsx


## É importantíssimo notar que após ETL, ainda fui manualmente buscar coordenadas específicas dada a falta de informação no Nomitatim 